In [3]:
from jsonschema import validate
import json

In [4]:
with open('examples/payload0.json') as infile:
    payload = json.load(infile)

In [5]:
payload

[{'applicants': [{'adjunct_income_eligibility': [{'program': 'snap',
      'verified': True},
     {'program': 'medicaid', 'verified': True}],
    'id': 1,
    'identity_proven': True,
    'physically_present': True},
   {'id': 2, 'identity_proven': True, 'physically_present': False},
   {'adjunct_income_eligibility': [{'program': 'snap',
      'verified': 'Excepted'}],
    'id': 3,
    'identity_proven': 'Excepted',
    'physically_present': True},
   {'id': 4, 'identity_proven': 'Excepted', 'physically_present': True}],
  'id': 1,
  'income': [{'dollars': 1480.5,
    'frequency': 'bi-weekly',
    'source': 'wages-and-salary',
    'verified': True},
   {'dollars': 1150.75,
    'frequency': 'weekly',
    'source': 'self-employment',
    'verified': False},
   {'dollars': 200,
    'frequency': 'semi-monthly',
    'source': 'social-security',
    'verified': 'Excepted'},
   {'dollars': 2000,
    'frequency': 'annually',
    'source': 'royalties',
    'verified': True},
   {'dollars': 200

In [6]:
validate(payload, {})

In [17]:
schema = {
    "$schema": "http://json-schema.org/draft-06/schema#",
    "title": "Application",
    "description": "A set of applications, one per household",
    "type": "array",
    "items": {
        "title": "applications",
        "type": "object",
            "properties": {
                "applicants": {
                "description": "Individuals (adults or children) to receive benefits",
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "id": {
                            "type": "integer",
                            "description": "Unique identifier within a submission",
                        },
                        "identity_proven": {       
                              "oneOf": [
                                { "type": "boolean",  },
                                { "type": "string", "value": "Excepted", }
                            ],
                            "description": "Has applicant met requirements for proving identity?",
                        }
                        
                    }
                        
                    }
                },
        }
    }
}

In [18]:
validate(payload, schema)

In [44]:
schema = {
    "$schema": "http://json-schema.org/draft-06/schema#",
    "title": "Application",
    "description": "A set of applications, one per household",
    
    "definitions": {
        "ynexception": {
                              "oneOf": [
                                { "type": "boolean",  },
                                { "type": "string", "value": "Excepted", }
                            ],
        }
    },
    "type": "array",
    "items": {
        "title": "applications",
        "type": "object",
            "properties": {
                "id": {
                    "type": "integer",
                },
                "number_in_economic_unit": {
                    "type": "integer",
                },
                "income": {
                    "type": "array",
                    "description": "All income sources for the entire economic unit / household",
                    "items": {
                        "type": "object",
                        "properties": {
                            "dollars": {
                                "type": "number",
                                "multipleOf": 0.01,
                            },
                            "source": {
                                "type": "string",
                                "description": "class of income",
                                "enum": ["wages-and-salary", "self-employment", "social-security", "interest-rent", 
                                        "assistance", "unemployment", "gov-retirement", "private-retirement",
                                        "alimony-and-child-support", "personal-contributions", "royalties",
                                        "other"],
                            },
                            "frequency": {
                                "type": "string",
                                "description": "How often `dollars` is received",
                                "enum": ['annual', 'monthly', 'semimonthly', 'semi-monthly',
                                         'twicemonthly', 'twice-monthly', 'biweekly', 'bi-weekly',
                                         'weekly'
                                ]
                            },
                            "verified": {
                                        "$ref": "#/definitions/ynexception",
                            }  
                            
                        }
                    }
                },
                "applicants": {
                "description": "Individuals (adults or children) to receive benefits",
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "id": {
                            "type": "integer",
                            "description": "Unique identifier within a submission",
                        },
                        "identity_proven": {      
                            "$ref": "#/definitions/ynexception",
                            "description": "Has applicant met requirements for proving identity?",
                        },
                        "physically_present": {      
                            "$ref": "#/definitions/ynexception",
                            "description": "Has applicant met requirements for proving identity?",
                        },     
                        "adjunct_income_eligibility": {
                            "description": "Any programs for which the applicant qualifies that confer automatic eligibility for this program.",
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "program": {
                                        "type": "string",
                                    },
                                    "verified": {
                                        "$ref": "#/definitions/ynexception",
                                    }
                                }
                            }
                            
                        }
                        
                    }
                        
                    }
                },
        }
    }
}

In [45]:
validate(payload, schema)

ValidationError: 'annually' is not one of ['annual', 'monthly', 'semimonthly', 'semi-monthly', 'twicemonthly', 'twice-monthly', 'biweekly', 'bi-weekly', 'weekly']

Failed validating 'enum' in schema['items']['properties']['income']['items']['properties']['frequency']:
    {'description': 'How often `dollars` is received',
     'enum': ['annual',
              'monthly',
              'semimonthly',
              'semi-monthly',
              'twicemonthly',
              'twice-monthly',
              'biweekly',
              'bi-weekly',
              'weekly'],
     'type': 'string'}

On instance[0]['income'][3]['frequency']:
    'annually'